In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout


data = yf.download("AAPL", start="2010-01-01", end="2023-01-01")
data = data[["Open"]]

scaler = MinMaxScaler()
scaled = scaler.fit_transform(data)

X, y = [], []
for i in range(60, len(scaled)):
    X.append(scaled[i-60:i])
    y.append(scaled[i])

X, y = np.array(X), np.array(y)

train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(60, 1)),
    Dropout(0.2),
    LSTM(50),
    Dropout(0.2),
    Dense(1)
])

model.compile(optimizer="adam", loss="mse")

model.fit(X_train, y_train, epochs=5, batch_size=32)

pred = model.predict(X_test)
pred = scaler.inverse_transform(pred)
real = scaler.inverse_transform(y_test)

plt.figure(figsize=(10,4))
plt.plot(real, label="Real Price")
plt.plot(pred, label="Predicted Price")
plt.legend()
plt.title("GRU Stock Price Prediction")
plt.show()
